In [4]:
import $file.common
import common._
import doobie._, doobie.implicits._
import cats.effect.IO

import $file.$     

import common._

import doobie._, doobie.implicits._

import cats.effect.IO

# Variation 3. Plain Old DAOs

So, we want to abstract the persistence layer from our business logic. The common answer to this problem, at least in the Java community, is based on [Data Object Access](https://en.wikipedia.org/wiki/Data_access_object) components. Basically, the persistence layer is isolated using conventional interfaces, as follows:

In [5]:
// Case classes

case class Country(code: String, name: String, capital: Option[Int])
case class City(id: Int, name: String, countryCode: String, population: Int)

// DAOs

trait CityDAO{
    def city(id: Int): City
    def cityName(id: Int): String
    def cityPopulation(id: Int): Int
    def cityCountryCode(id: Int): String
}

trait CountryDAO{
    def country(code: String): Country
    def countryName(code: String): String
    def countryCapital(code: String): Option[Int]
}

trait WorldDAO extends CityDAO with CountryDAO{
    def allCountries: List[Country]
    def allCountryCodes: List[String]
    def allCityIds: List[Int]
}

defined class Country
defined class City
defined trait CityDAO
defined trait CountryDAO
defined trait WorldDAO

Our query is implemented almost exactly as we did in the pure in-memory solution, only that the world dependency does not represent the actual data, but an interface to it:

In [6]:
def largeCities(implicit W: WorldDAO): List[(String, String)] = 
    for {
        Country(_, name, Some(capitalId)) <- W.allCountries
        city = W.city(capitalId) 
        if city.population > 8000000
    } yield (city.name, name)

defined function largeCities

This interface can be implemented the DAO interfaces so that they access the real world database through doobie:

In [7]:
class DoobieWorld(xa: Transactor[IO]) extends WorldDAO{
    def city(id: Int): City = 
        sql"select id, name, countryCode, population from city where id = $id"
            .query[City].unique.transact(xa).unsafeRunSync

    def cityName(id: Int): String =
        sql"select name from city where id = $id"
            .query[String].unique.transact(xa).unsafeRunSync
    
    def cityPopulation(id: Int): Int = 
        sql"select population from city where id = $id"
            .query[Int].unique.transact(xa).unsafeRunSync
    
    def cityCountryCode(id: Int): String = 
        sql"select countryCode from city where id = $id"
            .query[String].unique.transact(xa).unsafeRunSync

    def country(code: String): Country = 
        sql"select code, name, capital from country where code = $code"
            .query[Country].unique.transact(xa).unsafeRunSync
    
    def countryName(code: String): String =
        sql"select name from country where code = $code"
            .query[String].unique.transact(xa).unsafeRunSync
    
    def countryCapital(code: String): Option[Int] =
        sql"select capital from country where code = $code"
            .query[Int].option.transact(xa).unsafeRunSync
    
    def allCountries: List[Country] = 
        sql"select code, name, capital from country"
            .query[Country].to[List].transact(xa).unsafeRunSync
    
    def allCountryCodes: List[String] = 
        sql"select code from country"
            .query[String].to[List].transact(xa).unsafeRunSync
    
    def allCityIds: List[Int] = 
        sql"select id from city"
            .query[Int].to[List].transact(xa).unsafeRunSync
}

object DoobieWorld extends DoobieWorld(xa)

defined class DoobieWorld
defined object DoobieWorld

So that, we can query the actual database by injecting this dependency to our generic program:

In [8]:
largeCities(DoobieWorld)

res7: List[(String, String)] = List(
  ("Jakarta", "Indonesia"),
  ("Seoul", "South Korea"),
  ("Ciudad de M\u00e9xico", "Mexico"),
  ("Moscow", "Russian Federation")
)

Which other instances of DAO APIs we would like to have? Well, not exactly proper instances, but we may mock the APIs in order to unit test our business logic. So, great! We got idiomatic code, modularity (we may decompose the `largeCities` query in smaller query fragments), and unit testing. And, since we are programming against an abstract interface, we may later add new instances without breaking our code, in case that we eventually change our persistence layer, right?

### Asynchronous DAOs

Well, not exactly. The APIs we just wrote seems to be purely generic, but they unadvertently commit to a particular class of implementations, namely __synchronous__ ones. Indeed, there is no asynchronous instance of the previous APIs. So, if we want to benefit from accessing the database asynchronously, we have to change our API, and ... all the business logic that builds upon it. 

This asynchronous version of the DAO APIs may look like follows:

In [9]:
trait CityDAOAsync{
    def city(id: Int): IO[City]
    def cityName(id: Int): IO[String]
    def cityPopulation(id: Int): IO[Int]
    def cityCountryCode(id: Int): IO[String]
}

trait CountryDAOAsync{
    def country(code: String): IO[Country]
    def countryName(code: String): IO[String]
    def countryCapital(code: String): IO[Option[Int]]
}

trait WorldDAOAsync extends CityDAOAsync with CountryDAOAsync{
    def allCountries: IO[List[Country]]
    def allCountryCodes: IO[List[String]]
    def allCityIds: IO[List[Int]]
}

defined trait CityDAOAsync
defined trait CountryDAOAsync
defined trait WorldDAOAsync

Admittedly, we are committing ourselves to a particular IO monad, namely `cats.effect.IO`, but this doesn't matter, right? Well, chances are there that we may need to implement our DAOs using libraries based on alternative IO monads (`monix.Task`, `scalaz.Task`, `scala.concurrent.Future`, `ZIO`, etc.). That wouldn't be a stopper, but it would definitely require adaptor. Similarly, unit testing, which is far from being asynchronous, would have also to deal with IO. All this makes our code more complex than needed, and points towards a huge leak into our DAO abstractions. Can we do it better? Enter [MTL-based repositories](Variation4.MTL.ipynb)!